<a href="https://colab.research.google.com/github/Catisyf/Python-projects-chez-Toucan/blob/main/Backfill_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!ls '/content/drive/My Drive/crm_contacts.csv'

'/content/drive/My Drive/crm_contacts.csv'


In [16]:
import pandas as pd
from scipy import spatial

In [ ]:
pd_contacts = pd.read_csv('/content/drive/My Drive/crm_contacts.csv')
pd_persona = pd.read_csv('/content/drive/My Drive/buyer_persona_definition.csv')

In [15]:
pd_contacts

,jobtitle,email
0,Builder,joao.gonzalez@braskem.com
1,Singer-French songwriter,xlbjn@yahoo.es
2,Senior .NET Developer,kamilmrozek@gmail.com
3,Senior Analyst,slamcloud1@gmail.com
4,Finance Intern,osamaalqudah@mail.ru
...,...,...
25099,gmail (site),geay.isabelle@gmail.com
25100,gmail (site),maximin.daudiffret@gmail.com
25101,gmail (site),cindygros1@gmail.com
25102,gmail (site),ar.decision@gmail.com


In [13]:
pd_persona

,buyer_persona,job_title
0,"Data Leader - CIO, CDO",CIO
1,"Data Leader - CIO, CDO",CDO
2,"Data Leader - CIO, CDO",Chief Information Officer
3,"Data Leader - CIO, CDO",CIO
4,"Data Leader - CIO, CDO",Head of IT
...,...,...
126,Practitioner,Chef de Projet Senior BI
127,Practitioner,Chef de Projet Senior Business Intelligence
128,Practitioner,Chef de Projet Senior Insights
129,Practitioner,Chef de Projet Senior Analytics


In [ ]:
# Data preparation: remove null values 

In [32]:
#Sentence transformer model
pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.5 MB/s 
     |████████████████████████████████| 3.1 MB 11.0 MB/s 
     |████████████████████████████████| 3.3 MB 42.4 MB/s 
     |████████████████████████████████| 1.2 MB 54.6 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 60.5 MB/s 
     |████████████████████████████████| 895 kB 60.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=2becd3b83021b76679bd891df37d9361916349163bae449e7ca7ed3eb514a188
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [33]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/556 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/452 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [41]:
#Sentences are encoded by calling model.encode()
embeddings1 = model.encode('head marketing')
embeddings2 = model.encode('chef marketing')

In [40]:
print(embeddings1)

[ 2.86055189e-02  1.20746559e-02 -5.52894808e-02  7.09945988e-03
 -6.49541616e-05  2.68660579e-02  8.77373666e-03  6.60734400e-02
 -4.85596731e-02  2.80679055e-02 -3.03787179e-02  3.21675502e-02
  3.01978793e-02  4.31445092e-02  6.17392808e-02 -8.01867247e-03
  7.40744621e-02  3.30334604e-02  1.20462626e-02 -6.53400496e-02
 -8.47009942e-03  1.89355761e-02  2.54778899e-02 -5.37204370e-03
 -5.91953611e-03  1.40584195e-02  2.09833756e-02  1.82758085e-02
  8.63811560e-03 -2.66978201e-02 -2.54550036e-02  3.45224403e-02
 -2.55370010e-02 -2.58805379e-02 -6.65889010e-02 -6.99020401e-02
 -2.52816873e-03  5.31785376e-02 -2.67135482e-02 -5.19774780e-02
  5.50985113e-02  6.42504636e-03  2.37587113e-02 -4.94947284e-02
 -2.93991938e-02 -2.10980866e-02 -5.65067269e-02 -1.38607621e-02
  8.55476782e-02  1.17098391e-02  4.23826016e-02  2.85397694e-02
  7.49805942e-02 -7.14370087e-02  7.67917198e-04  1.78218242e-02
 -2.53335517e-02  7.76600018e-02 -2.83748619e-02  9.83559992e-03
  4.32200320e-02 -1.04414

In [69]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [70]:
import spacy

nlp = spacy.load("en_core_web_md")  # make sure to use larger package!
doc1 = nlp("I like salty fries and hamburgers.")
doc2 = nlp("Fast food tastes very good.")

OSError: ignored